# extract FITS image  

In [ ]:
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import Registry


In [ ]:
repo = 'embargo'
instrument = 'LSSTComCam'
dt = 'raw'
# raw all collection contains all images 
collections = [f'{instrument}/raw/all']
collections = ["LSSTComCam/defaults"]

butler = Butler(repo, collections=collections, instrument=instrument)

registry = butler.registry

#uncomment to get the help page
#help(registry) 

## Find the exposures
Listing the images could be done with butler using one of the tutorial queries.
The GEO observatons were taken on 2024 Dec 17, day_obs = 20241107, BLOCK-T332.
Keith mentioned there may be some other observations mixed in.

In [ ]:
day = 20241207
datasetType = 'raw'
block = 'BLOCK-T332'
dims = ['exposure']
result = registry.queryDataIds(  datasets=datasetType, dimensions = dims,
          where = f"day_obs = {day}  AND exposure.science_program='{block}'"
                              )
results = [r for r in result]

In [ ]:

print (f"got {len(results)} images")

ComCam has 9 images per exposure - this will list the exposures. 
The last digits after the daynumber  match the 'Seq. No.' in RubinTV

In [ ]:
exp=""
exposures = []
for r in results:
   if r['exposure'] != exp:
        exp=  r['exposure']
        exposures.append(exp)

print ("there are {len(exposures)} exposures")
print (exposures)


This is just getting the datasetref  the first exposure (this is comcam so that shoudl be 9 images one per detector)

In [ ]:
datasetRefs = registry.queryDatasets(datasetType=datasetType,
                                     where=f'exposure = {exposures[3]}')


## Write fits 
Write out fits files for the first count images. 
By default you will not have enough space to write all the files.

In [ ]:
images = []
for count, exp in enumerate(datasetRefs):
    fn=f"Rubin-{exp.dataId['exposure']}-{exp.dataId['detector']}.fits"
    img = butler.get('raw',exp.dataId)
    img.writeFits(fn)
    images.append(exp)
    
print (f"{count+1} images output")

Now we have Fits files we can use with external code.
We get the list of files we made and process them with SEP (Source Extractor Python) as an example of external code. 
We will make a list of Objects and store that.

In [ ]:
import glob
filelist = glob.glob('Rubin-*.fits')
print (filelist)

## If you want to plot one of these 

In [ ]:
import pylab as plt
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend('matplotlib')

for exp in datasetRefs:
    dataId=exp.dataId
    img = butler.get('raw', dataId=dataId)

    fig = plt.figure()
    display = afwDisplay.Display()
    display.scale('linear', 'zscale')
    display.mtv(img)
    plt.show()